In [0]:
!pip install kaggle --upgrade

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
import random
import os
import zipfile
from sklearn.model_selection import KFold
from PIL import Image 
from torchvision import datasets, transforms
import torchvision.models as tvm
from torchsummary import summary

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
! mkdir -p ~/.kaggle/
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! mkdir -p /content/competition/humpback-whale-identification
% cd /content/competition/humpback-whale-identification

!kaggle competitions download -c humpback-whale-identification

with zipfile.ZipFile("/content/competition/humpback-whale-identification/test.zip", 'r') as zip_ref:
  zip_ref.extractall("/content/competition/humpback-whale-identification/test")
with zipfile.ZipFile("/content/competition/humpback-whale-identification/train.zip", 'r') as zip_ref:
  zip_ref.extractall("/content/competition/humpback-whale-identification/train")

os.remove("test.zip")
os.remove("train.zip")

train_csv = pd.read_csv("/content/competition/humpback-whale-identification/train.csv")

In [0]:
def subset_data(num_count = 10,new_whale_count = 100):

  count_samples = train_csv.groupby('Id').agg('count')
  Id_subset = count_samples[count_samples['Image']>num_count]
  Id_subset_list = list(Id_subset.index)
  label_list = []

  for label in Id_subset_list:
    if (label != 'new_whale'):
      label_list.append(label)

  image_subset = train_csv[train_csv['Id'].isin(label_list)]
  image_new_whale = train_csv[train_csv['Id'] == 'new_whale']
  #image_new_whale_sample = image_new_whale.sample(n = new_whale_count)
  image_new_whale_sample = image_new_whale
  image_subset = image_subset.append(image_new_whale_sample)
  labels = pd.DataFrame(pd.unique(image_subset['Id']))
  labels['label_id'] = labels.index
  labels.columns = ['label','label_id']
  image_subset = pd.merge(image_subset,labels,left_on='Id',right_on='label',how = 'left')
  image_subset = image_subset[['Image','label_id']]
  image_subset = image_subset.sample(frac=1)
  image_label_list = image_subset.values.tolist()
  label_list = labels.values.tolist()
  count_labels = len(label_list)
  return image_label_list,count_labels,label_list

def val_generator(image_label_list,n_splits = 5):
  kf = KFold(n_splits=5,random_state = None)
  val_0,val_1,val_2,val_3,val_4 = kf.split(image_label_list)
  return val_0,val_1,val_2,val_3,val_4

## Image transforms

height = 256
width = 256

resize = transforms.Resize((height,width))
random_rotation = transforms.RandomRotation((-30,+30))
random_crop = transforms.RandomCrop((128,128),pad_if_needed=True)
horizontal_flip = transforms.transforms.RandomHorizontalFlip()
gray_scale = transforms.RandomGrayscale(p=0.5)
perspective = transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3)
noise = transforms.Compose([transforms.Lambda(lambda x : x + torch.randn_like(x))])

#normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
tensor_transform = transforms.ToTensor()

def train_image_transform(image):

  image = resize(image)
  #image = normalize(image)
  #image = horizontal_flip(image)
  image = gray_scale(image)
  image = perspective(image)
  if random.random() < 0.5:
    image = random_rotation(image)
  #image = random_crop(image)
  image = np.array(image)
  image = torch.tensor(image).float()
  image = image.permute(2,0,1)
  if random.random() < 0.5:
    image = noise(image)
  return image

def test_image_transform(image):

  image = resize(image)
  #image = normalize(image)
  image = np.array(image)
  image = torch.tensor(image).float()
  image = image.permute(2,0,1)
  return image

class WhaleTrainDataset(data.Dataset):

  def __init__(self,train_list, transform_train = None):
    super(WhaleTrainDataset, self).__init__()

    self.train_list = train_list
    self.no_of_samples = len(self.train_list)
    self.transform = transform_train

  def __getitem__(self,index):

    image_name = self.train_list[index][0]
    image = Image.open('/content/competition/humpback-whale-identification/train/{}'.format(image_name)).convert('RGB')
    label = self.train_list[index][1]
    image = self.transform_image(image)

    return image,label

  def __len__(self):
    return len(self.train_list)

  def transform_image(self,image):
    image = self.transform(image)
    return image

class WhaleValDataset (data.Dataset):

  def __init__(self,val_list,transform_val = None):
    super(WhaleValDataset, self).__init__()
    self.val_list = val_list
    self.no_of_samples = len(self.val_list)
    self.transform = transform_val

  def __getitem__(self,index):
    image_name = self.val_list[index][0]
    image = Image.open('/content/competition/humpback-whale-identification/train/{}'.format(image_name)).convert('RGB')
    label = self.val_list[index][1]
    image = self.transform_image(image)
    return image,label

  def __len__(self):
    return self.no_of_samples

  def transform_image(self,image):
    image = self.transform(image)
    return image

def train_valid_data_generator (val_fold):

  if val_fold == 0:
    train_index = val_0[0].tolist()
    val_index = val_0[1].tolist()
  elif val_fold == 1:
    train_index = val_1[0].tolist()
    val_index = val_1[1].tolist()
  elif val_fold == 2:
    train_index = val_2[0].tolist()
    val_index = val_2[1].tolist()
  elif val_fold == 3:
    train_index = val_3[0].tolist()
    val_index = val_3[1].tolist()
  elif val_fold == 4:
    train_index = val_4[0].tolist()
    val_index = val_4[1].tolist()

  train_list = [image_label_list[i] for i in train_index]
  val_list = [image_label_list[i] for i in val_index]

  return train_list,val_list

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):

  def load_pretrain(self, pretrain_file):

    pretrain_state_dict = torch.load(pretrain_file)
    state_dict = self.state_dict()
    keys = list(state_dict.keys())
    for key in keys:
      state_dict[key] = pretrain_state_dict[r'module.'+key]
      print(key)
    self.load_state_dict(state_dict)
    print('')

  def __init__(self,num_classes):

    super(Net,self).__init__()

    self.basemodel = tvm.resnet101(pretrained=True)
    self.basemodel.avgpool = nn.AdaptiveAvgPool2d(1)
    self.basemodel.last_layer = nn.Sequential()
    self.basemodel.layer0 = nn.Sequential(self.basemodel.conv1,
                                          self.basemodel.bn1,
                                          self.basemodel.relu,
                                          self.basemodel.maxpool) 

    self.fea_bn = nn.BatchNorm1d(2048)
    self.fc = nn.Linear(2048,num_classes)
    self.fea_bn.bias.requires_grad_(False)
  
  def forward(self,x):

    x = self.basemodel.layer0(x)
    x = self.basemodel.layer1(x)
    x = self.basemodel.layer2(x)
    x = self.basemodel.layer3(x)
    x = self.basemodel.layer4(x)

    x = self.basemodel.avgpool(x)
    fea = x.view(x.size(0),-1)
    fea = self.fea_bn(fea)

    output = (self.fc(fea))
    return output

def train_model(model,criterion,optimizer,scheduler,num_epochs):

  since = time.time()
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):

    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)

    for i in range(n_splits):
      
      print('Fold_index: {}'.format(i+1))

      train_list,val_list = train_valid_data_generator(val_fold = i)
      datasets = {'train':WhaleTrainDataset(train_list,train_image_transform),'val':WhaleValDataset(val_list,test_image_transform)}

      for phase in ['train','val']:
        if (phase=='train'):
          scheduler.step()
          model.train()
        else:
          model.eval()

        running_loss = 0.0
        running_corrects = 0
        
        dataset = datasets[phase]
        data_loader = data.DataLoader(dataset = dataset,batch_size=10)

        for images_tensor,labels in data_loader:

          images_tensor = images_tensor.to(device)
          labels = labels.to(device)
          optimizer.zero_grad()

          with torch.set_grad_enabled(phase=='train'):
            outputs = model(images_tensor)
            _,preds = torch.max(outputs,1)
            loss = criterion(outputs,labels)

            if(phase=='train'):
              loss.backward()
              optimizer.step()

          running_loss += loss.item() * images_tensor.size(0)
          running_corrects += torch.sum(preds == labels)

        phase_loss = running_loss / len(dataset)
        phase_acc = running_corrects.double() / len(dataset)
        
        model_name = 'model_resnet101_wt.tar'
        path = F"/content/gdrive/My Drive/{model_name}"
        torch.save({
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  }, path)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, phase_loss, phase_acc))

        if phase == 'val' and phase_acc > best_acc:
          best_acc = phase_acc
          best_model_wts = copy.deepcopy(model.state_dict())

    print()

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  model.load_state_dict(best_model_wts)
  return model 

In [0]:
image_label_list,count_labels,label_list = subset_data(num_count = 0)
val_0,val_1,val_2,val_3,val_4 = val_generator(image_label_list,n_splits = 5)

In [0]:
model = Net(num_classes = count_labels)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
exp_lr_scheduler = lr_scheduler.CyclicLR(optimizer_ft, base_lr = 0.000001 ,max_lr = 0.01)
num_epochs = 10
n_splits = 5

In [0]:
model_1 = train_model(model,criterion,optimizer_ft,exp_lr_scheduler,num_epochs)

In [0]:
##Training the model from the previous stopped epoch

model_name = 'model_resnet101_wt.tar'
path = F"/content/gdrive/My Drive/{model_name}"
checkpoint = torch.load(path)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer_ft.load_state_dict(checkpoint['optimizer_state_dict'])

model_1 = train_model(model,criterion,optimizer_ft,exp_lr_scheduler,num_epochs)